In [1]:
# def download_data():
#     !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
import numpy as np

In [3]:
def get_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

stored data moved to data folder using terminal

In [4]:
def one_time_preprocess(path):
    # read it in to inspect it
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
    print("length of dataset in characters: ", len(text))
    # let's look at the first 1000 characters
    print(text[:1000])
    print('here are all the unique characters that occur in this text')
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    print(''.join(chars))
    print("number of unique characters {}".format(vocab_size))
    return chars, vocab_size

In [5]:
chars, vocab_size = one_time_preprocess('data/input.txt')

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

***newline and space not visible in the list printed above

In [6]:
def create_maps_and_fxns(chars):
    # create a mapping from characters to integers
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(chars) }
    encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
    decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
    return encode, decode, stoi, itos

In [7]:
encode, decode, stoi, itos = create_maps_and_fxns(chars)

In [8]:
def test_encode_decode(text = "hii there"):
    print(encode(text))
    print(decode(encode(text)))

In [9]:
test_encode_decode()

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
def get_tensor_data(text):
    # let's now encode the entire text dataset and store it into a torch.Tensor
    data = torch.tensor(encode(text), dtype=torch.long)
    print(data.shape, data.dtype)
    #print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this
    return data

In [11]:
data = get_tensor_data(get_text('data/input.txt'))

torch.Size([1115394]) torch.int64


In [12]:
def get_train_test(data):
    # Let's now split up the data into train and validation sets
    n = int(0.9*len(data)) # first 90% will be train, rest val
    train_data = data[:n]
    val_data = data[n:]
    return train_data, val_data

In [13]:
train_data, val_data = get_train_test(data)

In [14]:
def get_batches(train_data, val_data):
    torch.manual_seed(1337)
    batch_size = 4 # how many independent sequences will we process in parallel?
    block_size = 8 # what is the maximum context length for predictions?

    def get_batch(split):
        # generate a small batch of data of inputs x and targets y
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        return x, y

    xb, yb = get_batch('train')
    print('inputs:')
    print(xb.shape)
    print(xb)
    print('targets:')
    print(yb.shape)
    print(yb)
    return xb, yb

In [15]:
xb, yb = get_batches(train_data, val_data)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
def visualize_batch(batch_size, block_size, xb, yb):
    for b in range(batch_size): # batch dimension
        for t in range(block_size): # time dimension
            context = xb[b, :t+1]
            target = yb[b,t]
            print(f"when input is {context.tolist()} the target: {target}")

In [17]:
visualize_batch(4,8,xb,yb)

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58
when input is [52, 58] the target: 1
when input is [52, 58, 1] the target: 58
when input is [52, 58, 1, 58] the target: 46
when input is [52, 58, 1, 58, 46] the target: 39
when input is [52, 58, 1, 58, 46, 39] the t

In [18]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [19]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward>)

lfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


In [20]:
# what is expected negative log likelihood loss?
-np.log(1/65)

4.174387269895637

In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [22]:
def get_batch(split):
        torch.manual_seed(1337)
        batch_size = 32 # how many independent sequences will we process in parallel?
        block_size = 8 # what is the maximum context length for predictions?
        # generate a small batch of data of inputs x and targets y
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        return x, y

In [23]:
x,y = get_batch('train')
print(x[31])
print(y[31])

tensor([53, 58,  6,  1, 51, 63,  1, 50])
tensor([58,  6,  1, 51, 63,  1, 50, 53])


In [24]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())



1.7060749530792236


In [25]:
def weighted_aggregation_sample():
    # consider the following toy example:

    torch.manual_seed(1337)
    B,T,C = 4,8,2 # batch, time, channels
    x = torch.randn(B,T,C)
    x.shape
    # We want x[b,t] = mean_{i<=t} x[b,i]
    xbow = torch.zeros((B,T,C))
    for b in range(B):
        for t in range(T):
            xprev = x[b,:t+1] # (t,C)
            xbow[b,t] = torch.mean(xprev, 0)
#     print(x[0])
#     print(xbow[0])
    # version 2: using matrix multiply for a weighted aggregation
#     wei = torch.tril(torch.ones(T, T))
#     wei = wei / wei.sum(1, keepdim=True)
#     xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
#     print(torch.allclose(xbow, xbow2))
    # version 3: use Softmax
    tril = torch.tril(torch.ones(T, T))
    wei = torch.zeros((T,T))
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    xbow3 = wei @ x
    print(torch.allclose(xbow, xbow3))

In [26]:
weighted_aggregation_sample()

True


In [29]:
# version 4: self-attention!
def sample_attn():
    torch.manual_seed(1337)
    B,T,C = 4,8,32 # batch, time, channels
    x = torch.randn(B,T,C)

    # let's see a single Head perform self-attention
    head_size = 16
    key = nn.Linear(C, head_size, bias=False)
    query = nn.Linear(C, head_size, bias=False)
    value = nn.Linear(C, head_size, bias=False)
    k = key(x)   # (B, T, 16)
    q = query(x) # (B, T, 16)
    wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

    tril = torch.tril(torch.ones(T, T))
    #wei = torch.zeros((T,T))
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    v = value(x)
    out = wei @ v
    #out = wei @ x

    print(out.shape)

In [30]:
sample_attn()

torch.Size([4, 8, 16])


Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [36]:
def visualize_init():
    '''the sqrt of key dimensions is done to control variance of the attention o/p and making it unit variance. 
    This is  cause softmax get peaky with when there is a large range of values and the o/p becomes more one-hot ish '''
    torch.manual_seed(1337)
    head_size = 16
    B,T,C = 4,8,32 # batch, time, channels
    k = torch.randn(B,T,head_size)
    q = torch.randn(B,T,head_size)
    wei = q @ k.transpose(-2, -1) * head_size**-0.5
    print(k.var())
    print(q.var())
    print(wei.var())
    print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
    print("O/p will be more one-hot ish")
    print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)) # gets too peaky, converges to one-hot

In [37]:
visualize_init()

tensor(1.0877)
tensor(1.0594)
tensor(1.1527)
tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
O/p will be more one-hot ish
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [38]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])